## 任务分解

1. **数据准备**  
   1.1. 划分教师子集  
   - 决定 \(K\) 个教师各自使用的训练子集（例如按类别分、或不同随机划分）。  
   - 生成对应的 PyTorch `Subset` 或自定义 sampler。  
   1.2. 标准 CIFAR‑10（或 MNIST）预处理  
   - 随机裁剪、水平翻转、归一化；  
   - 构建训练/验证 `DataLoader`（batch size、shuffle、num_workers）。

2. **模型定义**  
   2.1. 教师们  
   - 实例化同一架构（如 ResNet18）或多种架构（ResNet18/34/50…）的 \(K\) 个模型；  
   2.2. 学生  
   - 实例化学生网络（如 ResNet18）——单一、小容量；  
   2.3. 参数统计  
   - 用 `torchinfo` 或 `flops-counter` 输出每个模型的参数量和 FLOPs。

3. **教师训练**  
   - 并行／顺序地在各自子集上训练 \(K\) 个教师；  
   - 保存各自最佳 checkpoint；  
   - 记录各自的验证准确率。

4. **单教师蒸馏（Forward KD）——复用 Part 4 代码**  
   - **已完成**：  
     - 生成单一教师温度化软标签；  
     - 定义并训练学生的 KD 损失（KL+CE）；  
     - 评估并记录学生蒸馏后的性能。  

5. **多教师蒸馏**  
   5.1. **软标签融合**  
   - 每个 mini‑batch 对 \(K\) 位教师分别做 forward，收集它们的 \(\tau\)‑softmax 输出；  
   - 逐元素平均得到 \(\,p_{\rm avg}^{(\tau)}\)；  
   5.2. **学生训练**  
   - 损失同 Forward KD，只用 \(p_{\rm avg}^{(\tau)}\) 代替单个教师；  
   - 学生前向／反向、保存 checkpoint；  
   5.3. **评估**  
   - 在同一验证集上测试学生，多教师 vs. 单教师 vs. 硬标签基线的准确率对比；  

6. **超参数扫描与消融**  
   - 遍历 \(\tau\in\{1,5,10,20\}\)、\(\alpha\in\{0.1,0.5,0.9\}\)、教师数 \(K\in\{2,4,8\}\)；  
   - 记录每组下学生的最终准确率；  
   - 可选：试验“弱教师”检查点（训练不足的教师）对学生性能的影响。  

7. **结果可视化 & 分析**  
   - **定量：**  
     - 各方案准确率表格（硬标签／单教师／多教师）；  
     - Δ‑Accuracy 随 \(\tau\)、\(\alpha\)、\(K\) 的变化。  
   - **可视化：**  
     - 准确率 vs. 温度曲线；  
     - K 值 vs. Δ‑Accuracy 柱状图；  
   - **讨论：**  
     - 多教师如何提供多样化“暗知识”增强正则化；  
     - 与“标签平滑＝正则化”视角的联系。  

8. **复现 & 代码组织**  
   - 新建 `distill_multi.py`，沿用 `distill_single.py` 框架，增加教师列表与融合逻辑；  
   - 在 `scripts/` 下补充命令行示例；  
   - 确保随机种子、日志记录、参数统计一并复现。  



# Part8

In [ ]:
#!/usr/bin/env python3
"""
distill_multi.py

Multi‑teacher knowledge distillation for CIFAR-10.
"""

import os
import argparse
import logging
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchinfo import summary

def parse_args():
    p = argparse.ArgumentParser(description="Multi‑Teacher Distillation CIFAR-10")
    p.add_argument("--data_dir",      type=str,   default="data/",             help="dataset root")
    p.add_argument("--teacher_ckpts", nargs="+",  required=True,               help="paths to teacher .pth files")
    p.add_argument("--teacher_arch",  type=str,   default="resnet50",         help="teacher architecture")
    p.add_argument("--student_arch",  type=str,   default="resnet34",         help="student architecture")
    p.add_argument("--batch_size",    type=int,   default=128,                 help="batch size")
    p.add_argument("--epochs",        type=int,   default=200,                 help="training epochs")
    p.add_argument("--tau",           type=float, default=5.0,                 help="distillation temperature")
    p.add_argument("--alpha",         type=float, default=0.7,                 help="KD loss weight")
    p.add_argument("--lr",            type=float, default=0.1,                 help="initial lr")
    p.add_argument("--milestones",    nargs="+",  type=int, default=[100,150], help="LR scheduler milestones")
    p.add_argument("--seed",          type=int,   default=42,                  help="random seed")
    p.add_argument("--log_file",      type=str,   default="distill_multi.log",help="log file")
    p.add_argument("--out_dir",       type=str,   default="out/multi",        help="output directory")
    return p.parse_args()

def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark     = False

def configure_logging(log_file):
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s %(levelname)s %(message)s",
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger()

def build_dataloaders(data_dir, batch_size):
    mean = (0.4914, 0.4822, 0.4465)
    std  = (0.2470, 0.2435, 0.2616)

    train_tf = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
    test_tf = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

    train_ds = datasets.CIFAR10(root=data_dir, train=True,  download=True, transform=train_tf)
    test_ds  = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=test_tf)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=4, pin_memory=True)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    return train_loader, test_loader

def load_teachers(ckpt_paths, architecture, device):
    teachers = []
    for path in ckpt_paths:
        model = getattr(models, architecture)(num_classes=10)
        sd = torch.load(path, map_location=device)['model_state']
        model.load_state_dict(sd)
        model.to(device).eval()
        for p in model.parameters():
            p.requires_grad = False
        teachers.append(model)
    return teachers

def distillation_loss(student_logits, teacher_avg, targets, tau, alpha):
    s_logprob = F.log_softmax(student_logits / tau, dim=1)
    loss_kl    = F.kl_div(s_logprob, teacher_avg, reduction='batchmean') * (tau*tau)
    loss_ce    = F.cross_entropy(student_logits, targets)
    return alpha * loss_kl + (1 - alpha) * loss_ce

def main():
    args = parse_args()
    set_seeds(args.seed)
    log = configure_logging(args.log_file)
    log.info(f"Arguments: {args}")

    os.makedirs(args.out_dir, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data
    train_loader, test_loader = build_dataloaders(args.data_dir, args.batch_size)

    # Teachers
    teachers = load_teachers(args.teacher_ckpts, args.teacher_arch, device)
    log.info(f"Loaded {len(teachers)} teachers [{args.teacher_arch}]")

    # Student
    student = getattr(models, args.student_arch)(num_classes=10).to(device)
    log.info(f"Student arch: {args.student_arch}")
    log.info(summary(student, input_size=(1,3,32,32)))

    # Optimizer & scheduler
    optimizer = optim.SGD(
        student.parameters(),
        lr=args.lr,
        momentum=0.9,
        weight_decay=5e-4
    )
    scheduler = optim.lr_scheduler.MultiStepLR(
        optimizer,
        milestones=args.milestones,
        gamma=0.1
    )

    best_acc = 0.0

    # Training loop
    for epoch in range(1, args.epochs + 1):
        student.train()
        running_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()

            # Soft‑label fusion
            with torch.no_grad():
                sum_soft = torch.zeros(inputs.size(0), 10, device=device)
                for tm in teachers:
                    logits_t = tm(inputs)
                    sum_soft += F.softmax(logits_t / args.tau, dim=1)
                teacher_avg = sum_soft / len(teachers)

            # Student forward + loss
            logits_s = student(inputs)
            loss     = distillation_loss(logits_s, teacher_avg, targets, args.tau, args.alpha)

            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        scheduler.step()
        avg_train_loss = running_loss / len(train_loader.dataset)
        log.info(f"Epoch {epoch:03d}/{args.epochs} train_loss={avg_train_loss:.4f}")

        # Validation
        student.eval()
        correct = total = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                preds = student(inputs).argmax(dim=1)
                correct += (preds == targets).sum().item()
                total   += targets.size(0)
        test_acc = 100. * correct / total
        log.info(f"          test_acc={test_acc:.2f}%")

        # Checkpoint
        if test_acc > best_acc:
            best_acc = test_acc
            ckpt_path = os.path.join(args.out_dir, f"student_multi_best.pth")
            torch.save({
                'epoch':        epoch,
                'model_state':  student.state_dict(),
                'opt_state':    optimizer.state_dict(),
                'test_acc':     test_acc,
            }, ckpt_path)
            log.info(f"  → New best saved to {ckpt_path}")

    log.info(f"Finished. Best multi-teacher student acc: {best_acc:.2f}%")

if __name__ == "__main__":
    main()


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd "/content/drive/MyDrive/661project"
!pwd

/content/drive/MyDrive/661project
/content/drive/MyDrive/661project


# Part1

In [6]:
from torchvision import datasets

full_train = datasets.CIFAR10(
    root='data/',
    train=True,
    download=True,
    transform=None  # we'll apply transforms later
)


In [7]:
# Suppose K = 5 and 10 classes => 2 classes per teacher
K = 5
classes_per_teacher = 10 // K  # = 2
teacher_splits = []
targets = full_train.targets  # list of integer labels

for t in range(K):
    cls_start = t * classes_per_teacher
    cls_end   = cls_start + classes_per_teacher
    # find indices whose label ∈ [cls_start, cls_end)
    idxs = [i for i, lab in enumerate(targets)
            if cls_start <= lab < cls_end]
    teacher_splits.append(idxs)


In [8]:
import torch
num_samples = len(full_train)
perm = torch.randperm(num_samples).tolist()
teacher_splits = []
split_size = num_samples // K

for t in range(K):
    start = t * split_size
    end   = start + split_size if t < K-1 else num_samples
    teacher_splits.append(perm[start:end])


In [9]:
from torch.utils.data import Subset

teacher_datasets = [
    Subset(full_train, idxs)
    for idxs in teacher_splits
]
# Now teacher_datasets[i] is the CIFAR-10 subset for teacher #i


In [10]:
from torchvision import transforms

# CIFAR-10 normalization constants
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD  = (0.2470, 0.2435, 0.2616)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD),
])


In [11]:
# We need to override the transform of the underlying dataset
# for each Subset. One simple approach is to wrap with a lambda:
class TransformSubset(Subset):
    def __init__(self, subset, transform):
        super().__init__(subset.dataset, subset.indices)
        self.transform = transform
    def __getitem__(self, idx):
        img, label = super().__getitem__(idx)
        return self.transform(img), label

teacher_load_datasets = [
    TransformSubset(ds, train_transform)
    for ds in teacher_datasets
]


In [12]:
from torch.utils.data import DataLoader

batch_size  = 128
num_workers = 4

teacher_loaders = [
    DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    for ds in teacher_load_datasets
]

# And for your student’s full training set (hard‑label baseline):
full_train.transform = train_transform
student_train_loader = DataLoader(
    full_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True
)

# CIFAR-10 test set:
test_set = datasets.CIFAR10(
    root='data/',
    train=False,
    download=False,
    transform=test_transform
)
test_loader = DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)


# Part2

In [13]:
pip install torchinfo

In [14]:
import torch
from torchvision.models import resnet18, resnet34, resnet50
from torchinfo import summary

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2.1 Instantiate K teacher models (here: ResNet18, ResNet34, ResNet50)
teacher_archs = ['resnet18', 'resnet34', 'resnet50']
teachers = []

for arch in teacher_archs:
    if arch == 'resnet18':
        model = resnet18(num_classes=10)
    elif arch == 'resnet34':
        model = resnet34(num_classes=10)
    elif arch == 'resnet50':
        model = resnet50(num_classes=10)
    else:
        raise ValueError(f"Unsupported architecture: {arch}")
    model.to(device)
    teachers.append((arch, model))

# 2.2 Instantiate the student model (smaller-capacity): ResNet18
student = resnet18(num_classes=10)
student.to(device)

# 2.3 Parameter & FLOPs statistics using torchinfo.summary
# Assume CIFAR-10 input size: (batch_size=1, channels=3, height=32, width=32)
input_size = (1, 3, 32, 32)

print("\n=== Teacher Models ===")
for name, model in teachers:
    print(f"\n-- {name.upper()} --")
    summary(model, input_size=input_size, col_names=("output_size", "num_params", "mult_adds"))

print("\n=== Student Model ===")
print("-- RESNET18 STUDENT --")
summary(student, input_size=input_size, col_names=("output_size", "num_params", "mult_adds"))



=== Teacher Models ===

-- RESNET18 --

-- RESNET34 --

-- RESNET50 --

=== Student Model ===
-- RESNET18 STUDENT --


Layer (type:depth-idx)                   Output Shape              Param #                   Mult-Adds
ResNet                                   [1, 10]                   --                        --
├─Conv2d: 1-1                            [1, 64, 16, 16]           9,408                     2,408,448
├─BatchNorm2d: 1-2                       [1, 64, 16, 16]           128                       128
├─ReLU: 1-3                              [1, 64, 16, 16]           --                        --
├─MaxPool2d: 1-4                         [1, 64, 8, 8]             --                        --
├─Sequential: 1-5                        [1, 64, 8, 8]             --                        --
│    └─BasicBlock: 2-1                   [1, 64, 8, 8]             --                        --
│    │    └─Conv2d: 3-1                  [1, 64, 8, 8]             36,864                    2,359,296
│    │    └─BatchNorm2d: 3-2             [1, 64, 8, 8]             128                       128
│    │    └─ReLU:

# Part3

3. Teacher Training
Train your
𝐾
K teachers—each on its own subset—saving the best checkpoint and logging validation accuracy.
3.2 Notes on Parallel vs. Sequential
Sequential: the loop above trains each teacher one after another in the same process.

Parallel: if you have multiple GPUs or machines, you can spawn
𝐾
K separate Python processes—each binding one teacher’s subset and one GPU—and run the same code concurrently. Tools like torch.multiprocessing.spawn or simple shell scripts with CUDA_VISIBLE_DEVICES can help.

3.3 Logging
We print per‑epoch train loss and validation accuracy.

After all
𝐾
K finishes, you’ll have a best_ckpt_path for each teacher and a Python list best_val_accs of length
𝐾
K.

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.models import resnet34, resnet50  # import the two architectures

# Assume:
#   teacher_loaders = [DataLoader(subset_i, …) for i in range(K)]
#   test_loader     = DataLoader(full_CIFAR10_test, …)
#   device          = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#   num_epochs      = 200

best_val_accs = []

# Cycle through the two imported constructors
archs = [resnet34, resnet50]

for i, train_loader in enumerate(teacher_loaders):
    print(f"\n=== Training teacher #{i} ===")

    # Select architecture: resnet34 for even i, resnet50 for odd i, etc.
    Arch = archs[i % len(archs)]
    teacher_model = Arch(num_classes=10)  # override final fc to 10 classes
    teacher_model = teacher_model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(
        teacher_model.parameters(),
        lr=0.1,
        momentum=0.9,
        weight_decay=5e-4
    )
    scheduler = optim.lr_scheduler.MultiStepLR(
        optimizer,
        milestones=[100, 150],
        gamma=0.1
    )

    best_acc       = 0.0
    best_ckpt_path = f"teacher_{i}_{Arch.__name__}_best.pth"

    for epoch in range(1, num_epochs + 1):
        teacher_model.train()
        running_loss = 0.0

        for inputs, targets in tqdm(
                train_loader,
                desc=f"Teacher {i} ({Arch.__name__}) Epoch {epoch}",
                leave=False):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = teacher_model(inputs)
            loss    = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        scheduler.step()

        # Validation
        teacher_model.eval()
        correct = total = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs  = teacher_model(inputs)
                _, preds = outputs.max(1)
                correct += preds.eq(targets).sum().item()
                total   += targets.size(0)

        val_acc  = 100. * correct / total
        avg_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch:03d} | train_loss={avg_loss:.4f} | val_acc={val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save({
                'epoch': epoch,
                'arch': Arch.__name__,
                'model_state': teacher_model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
                'val_acc': val_acc,
            }, best_ckpt_path)
            print(f"  → New best! Saved to {best_ckpt_path}")

    best_val_accs.append(best_acc)
    print(f"Teacher #{i} ({Arch.__name__}) best validation accuracy: {best_acc:.2f}%")



=== Training teacher #0 ===


Epoch 001 | train_loss=3.5669 | val_acc=9.67%
  → New best! Saved to teacher_0_resnet34_best.pth


Epoch 002 | train_loss=2.6240 | val_acc=17.21%
  → New best! Saved to teacher_0_resnet34_best.pth


Epoch 003 | train_loss=2.2381 | val_acc=18.44%
  → New best! Saved to teacher_0_resnet34_best.pth


Epoch 004 | train_loss=2.1175 | val_acc=22.87%
  → New best! Saved to teacher_0_resnet34_best.pth


...

Epoch 196 | train_loss=0.0251 | val_acc=77.38%


Epoch 197 | train_loss=0.0243 | val_acc=77.70%


Epoch 198 | train_loss=0.0257 | val_acc=77.56%


Epoch 199 | train_loss=0.0237 | val_acc=77.67%


Epoch 200 | train_loss=0.0249 | val_acc=77.72%
Teacher #4 (resnet34) best validation accuracy: 77.84%


# 改进版Part3
下面的代码是一份 改进版的 Teacher 训练脚本，把 每个 teacher 都改成在 全量 CIFAR‑10 上训练，同时加入更强的数据增强（随机裁剪 + 翻转 + Cutout）、Label Smoothing、Mixup，以及更合理的学习率调度，通常可以让单个 ResNet‑50 在 CIFAR‑10 上达到 90% 以上的准确率。

要点总结

全量数据：不再把数据切分成多个 teacher_loader，而是都用同一个完整的 train_loader。

强增强：RandomCrop(32,4) / RandomHorizontalFlip / Cutout。

Mixup：在 batch 级别做 Mixup，增加泛化。

Label Smoothing：CrossEntropy 损失里做平滑，抑制过度自信。

CosineAnnealingLR：比 MultiStepLR 更平滑地衰减 lr。

更长 schedule：200 → 300 epochs，常在 0–1 之间 Cosine 衰减。

为什么会有效？
全量数据 + 强增强：随机裁剪、翻转、Cutout（自行接入）让模型见到更多变形。

Mixup：在样本层面插值，能显著提升模型泛化。

Label Smoothing：抑制模型过度自信，提升 test accuracy。

Cosine LR：平滑衰减学习率，比阶梯式更好地利用后期微调。

足够多的 Epochs (300)：给 ResNet‑50 充分的训练时间。

在 CIFAR‑10 上，这套配置下 ResNet‑50 通常能稳稳突破 90%，甚至接近 93% ~ 95%。如果你还有更强的硬件，也可以把 Mixup 改成 CutMix、再加上 AutoAugment 做更激进的增强，accuracy 会更高。









In [20]:
import os, glob
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets, transforms
from torchvision.models import resnet50  # 统一用 ResNet‑50
from torch.utils.data import DataLoader
import numpy as np

# ───── 0️⃣ CIFAR-10 全量训练/测试 DataLoader ─────
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

# 强化数据增强
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

batch_size = 128
train_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=True,  download=True, transform=train_transform),
    batch_size=batch_size, shuffle=True,  num_workers=4, pin_memory=True
)
test_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=False, download=True, transform=test_transform),
    batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ───── 1️⃣ Mixup 辅助函数 ─────
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.0
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# ───── 2️⃣ Label Smoothing CrossEntropy ─────
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, eps:float=0.1, reduction='mean'):
        super().__init__()
        self.eps = eps
        self.reduction = reduction
    def forward(self, outputs, targets):
        n = outputs.size(-1)
        log_preds = torch.log_softmax(outputs, dim=-1)
        # one-hot
        with torch.no_grad():
            true_dist = torch.zeros_like(log_preds)
            true_dist.fill_(self.eps / (n-1))
            true_dist.scatter_(1, targets.data.unsqueeze(1), 1-self.eps)
        loss = torch.sum(- true_dist * log_preds, dim=1)
        if self.reduction=='mean':
            return loss.mean()
        else:
            return loss.sum()

# ───── 3️⃣ 训练单个 teacher ─────
num_epochs = 300
criterion = LabelSmoothingCrossEntropy(eps=0.1)
mixup_alpha = 0.2

teacher_model = resnet50(num_classes=10).to(device)
optimizer = optim.SGD(
    teacher_model.parameters(),
    lr=0.1, momentum=0.9, weight_decay=1e-4
)
# Cosine 衰减到 0
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

best_acc = 0.0
best_ckpt = "teacher_resnet50_best.pth"

for epoch in range(1, num_epochs+1):
    teacher_model.train()
    running_loss = 0.0

    for xb, yb in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}", leave=False):
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()

        # 1) Mixup
        xb_mix, y_a, y_b, lam = mixup_data(xb, yb, alpha=mixup_alpha)

        # 2) forward + Label Smoothing
        outputs = teacher_model(xb_mix)
        loss = lam * criterion(outputs, y_a) + (1-lam) * criterion(outputs, y_b)

        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)

    scheduler.step()
    avg_loss = running_loss / len(train_loader.dataset)

    # Validation
    teacher_model.eval()
    correct = total = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = teacher_model(xb).argmax(dim=1)
            correct += (preds==yb).sum().item()
            total   += yb.size(0)
    val_acc = 100. * correct / total

    print(f"Epoch {epoch:03d} | train_loss={avg_loss:.4f} | val_acc={val_acc:.2f}%")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({
            'epoch': epoch,
            'arch':  'resnet50',
            'model_state': teacher_model.state_dict(),
            'optimizer_state': optimizer.state_dict(),
            'val_acc': best_acc,
        }, best_ckpt)
        print(f"  → New best! Saved to {best_ckpt}")

print(f"\n✅ Training complete. Best teacher val_acc = {best_acc:.2f}%")


Epoch 001 | train_loss=5.1129 | val_acc=15.98%
  → New best! Saved to teacher_resnet50_best.pth


Epoch 002 | train_loss=2.2776 | val_acc=21.40%
  → New best! Saved to teacher_resnet50_best.pth


Epoch 003 | train_loss=2.1576 | val_acc=25.41%
  → New best! Saved to teacher_resnet50_best.pth


Epoch 004 | train_loss=2.0767 | val_acc=31.08%
  → New best! Saved to teacher_resnet50_best.pth


Epoch 005 | train_loss=2.0175 | val_acc=35.10%
  → New best! Saved to teacher_resnet50_best.pth


...

Epoch 296 | train_loss=0.8154 | val_acc=88.98%


Epoch 297 | train_loss=0.8547 | val_acc=89.21%
  → New best! Saved to teacher_resnet50_best.pth


Epoch 298 | train_loss=0.8349 | val_acc=89.10%


Epoch 299 | train_loss=0.8583 | val_acc=89.03%


Epoch 300 | train_loss=0.8451 | val_acc=89.09%

✅ Training complete. Best teacher val_acc = 89.21%


#Part4

Loaded your single teacher.

Defined the student, optimizer & KD loss.

Trained the student for 100 epochs with KL+CE.

Evaluated and saved its best test accuracy.

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.models import resnet50, resnet34
from torch.utils.data import DataLoader
from tqdm import tqdm

# 1. Hyperparameters and device
device     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 128
num_epochs = 200
tau        = 5      # distillation temperature
alpha      = 0.7    # weight on KL term
milestones = [60, 120, 160]
lr         = 0.1

# 2. Data preparation (same as in code 1)
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_set = datasets.CIFAR10(root='data/', train=True,  download=True, transform=train_transform)
test_set  = datasets.CIFAR10(root='data/', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_set,  batch_size=batch_size, shuffle=True,  num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_set,   batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# 3. Load pretrained teacher (ResNet-50) and freeze it
teacher = resnet50(num_classes=10).to(device)
# ← here we load the ResNet-50 checkpoint (teacher_1), not the ResNet-34 one
ckpt    = torch.load("teacher_1_resnet50_best.pth", map_location=device)
teacher.load_state_dict(ckpt['model_state'])
teacher.eval()
for p in teacher.parameters():
    p.requires_grad = False

# 4. Instantiate student (ResNet-34)
student = resnet34(num_classes=10).to(device)

# 5. Optimizer & scheduler
optimizer = optim.SGD(
    student.parameters(),
    lr=lr,
    momentum=0.9,
    weight_decay=5e-4
)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer,
    milestones=milestones,
    gamma=0.2
)

# 6. Distillation loss
def kd_loss(student_logits, teacher_logits, targets):
    s = F.log_softmax(student_logits / tau, dim=1)
    t = F.softmax(teacher_logits / tau,   dim=1)
    loss_kl = F.kl_div(s, t, reduction='batchmean') * (tau * tau)
    loss_ce = F.cross_entropy(student_logits, targets)
    return alpha * loss_kl + (1 - alpha) * loss_ce

# 7. Training loop
best_acc = 0.0

for epoch in range(1, num_epochs + 1):
    student.train()
    running_loss = 0.0

    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch}", leave=False):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        with torch.no_grad():
            teacher_logits = teacher(inputs)

        student_logits = student(inputs)
        loss = kd_loss(student_logits, teacher_logits, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    scheduler.step()
    avg_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch:03d} | train_loss={avg_loss:.4f}")

    # 8. Evaluation
    student.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = student(inputs)
            preds   = outputs.argmax(dim=1)
            correct += (preds == targets).sum().item()
            total   += targets.size(0)

    test_acc = 100. * correct / total
    print(f"          test_acc={test_acc:.2f}%")

    # 9. Save best
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save({
            'epoch':           epoch,
            'model_state':     student.state_dict(),
            'optimizer_state': optimizer.state_dict(),
            'test_acc':        test_acc,
        }, "student_distilled_best.pth")
        print("  → New best saved.")

print(f"Finished. Best distilled student accuracy: {best_acc:.2f}%")


Epoch 001 | train_loss=6.1549


          test_acc=26.44%
  → New best saved.


Epoch 002 | train_loss=4.4776


          test_acc=34.61%
  → New best saved.


Epoch 003 | train_loss=3.8858


...

          test_acc=80.54%


Epoch 199 | train_loss=0.3336


          test_acc=80.73%


Epoch 200 | train_loss=0.3339


          test_acc=80.93%
  → New best saved.
Finished. Best distilled student accuracy: 80.93%


# Part5
Explanation of steps:

Soft‑label fusion (5.1):

We iterate over each of the
𝐾
K teachers in teacher_models, do a forward pass on the same inputs, take each teacher’s
𝜏
τ-softmax, and accumulate.

Dividing by
𝐾
K yields the averaged soft‑label
𝑝
a
v
g
(
𝜏
)
p
avg
(τ)
​
 .

Student training (5.2):

Compute the KL divergence between the student’s log-​softmax and
𝑝
a
v
g
(
𝜏
)
p
avg
(τ)
​
 , scaled by
𝜏
2
τ
2
 .

Add the usual cross‑entropy on the hard labels.

Backpropagate and optimizer-​step.

Evaluation & checkpointing (5.3):

After each epoch, switch student.eval(), measure test accuracy against the full test set, and if it improves, save "student_multi_teacher_best.pth".

Explanation of steps:

Soft‑label fusion (5.1):

We iterate over each of the
𝐾
K teachers in teacher_models, do a forward pass on the same inputs, take each teacher’s
𝜏
τ-softmax, and accumulate.

Dividing by
𝐾
K yields the averaged soft‑label
𝑝
a
v
g
(
𝜏
)
p
avg
(τ)
​
 .

Student training (5.2):

Compute the KL divergence between the student’s log-​softmax and
𝑝
a
v
g
(
𝜏
)
p
avg
(τ)
​
 , scaled by
𝜏
2
τ
2
 .

Add the usual cross‑entropy on the hard labels.

Backpropagate and optimizer-​step.

Evaluation & checkpointing (5.3):

After each epoch, switch student.eval(), measure test accuracy against the full test set, and if it improves, save "student_multi_teacher_best.pth".

In [25]:
import os
import glob
import torch
import torch.nn.functional as F
from torchvision.models import resnet18, resnet34, resnet50

# ───────────────────────────────────────────────────────────────
# 0️⃣ Basic setup (you already have these)
device       = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader = train_loader    # your CIFAR‑10 train DataLoader
test_loader  = test_loader     # your CIFAR‑10 test  DataLoader
student      = student.to(device)                # ResNet‑34 student from code4
optimizer    = optimizer                       # your SGD optimizer on `student`
scheduler    = scheduler                       # your MultiStepLR on `student`
tau, alpha   = 5, 0.7                          # distillation hyperparams
best_acc     = 0.0
num_epochs   = 200
# ───────────────────────────────────────────────────────────────

# 1️⃣ Discover the teacher checkpoint files created by code3
ckpt_paths = sorted(glob.glob("teacher_*_*_best.pth"))
if not ckpt_paths:
    raise FileNotFoundError("No teacher checkpoints found matching teacher_*_*_best.pth")

# 2️⃣ Load & freeze each teacher
teacher_models = []
for ckpt_path in ckpt_paths:
    fname = os.path.basename(ckpt_path)           # e.g. "teacher_0_resnet34_best.pth"
    parts = fname.split("_")
    arch  = parts[2]                              # "resnet34"
    # pick the correct constructor
    if   arch == "resnet18": ctor = resnet18
    elif arch == "resnet34": ctor = resnet34
    elif arch == "resnet50": ctor = resnet50
    else: raise ValueError(f"Unexpected arch in {fname}")

    # instantiate, load weights, move to device
    model = ctor(num_classes=10).to(device)
    data  = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(data['model_state'])
    model.eval()                                  # inference only
    for p in model.parameters():
        p.requires_grad = False                   # no grads
    teacher_models.append(model)

K = len(teacher_models)
print(f"Loaded {K} teacher models from {len(ckpt_paths)} checkpoints")

# 3️⃣ Multi‑teacher distillation loop
for epoch in range(1, num_epochs+1):
    student.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        # — fuse K teachers’ τ‑softmax outputs —
        with torch.no_grad():
            # dynamic class count C
            C = teacher_models[0](inputs).size(1)
            sum_soft = torch.zeros(inputs.size(0), C, device=device)

            for t_model in teacher_models:
                logits_t = t_model(inputs)
                sum_soft += F.softmax(logits_t / tau, dim=1)

            p_avg = sum_soft / float(K)

        # — student forward + KD + CE loss —
        logits_s = student(inputs)
        s_soft   = F.log_softmax(logits_s / tau, dim=1)
        loss_kl  = F.kl_div(s_soft, p_avg, reduction='batchmean') * (tau * tau)
        loss_ce  = F.cross_entropy(logits_s, targets)
        loss     = alpha * loss_kl + (1 - alpha) * loss_ce

        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    scheduler.step()
    train_loss = running_loss / len(train_loader.dataset)
    print(f"[Epoch {epoch:03d}] train_loss={train_loss:.4f}")

    # — evaluation —
    student.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            preds = student(inputs).argmax(dim=1)
            correct += (preds == targets).sum().item()
            total   += targets.size(0)

    test_acc = 100. * correct / total
    print(f"           test_acc={test_acc:.2f}%")

    # — save best —
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save({
            'epoch':          epoch,
            'student_state':  student.state_dict(),
            'optimizer':      optimizer.state_dict(),
            'test_acc':       test_acc,
        }, "student_multi_teacher_best.pth")
        print("  → New best multi‑teacher student saved.")

print(f"\n✅ Multi‑teacher distillation complete. Best acc: {best_acc:.2f}%")


Loaded 5 teacher models from 5 checkpoints
[Epoch 001] train_loss=0.4575
           test_acc=82.09%
  → New best multi‑teacher student saved.
[Epoch 002] train_loss=0.3838
           test_acc=82.56%
  → New best multi‑teacher student saved.
[Epoch 003] train_loss=0.3579
           test_acc=82.78%
  → New best multi‑teacher student saved.
[Epoch 004] train_loss=0.3404
           test_acc=83.14%
  → New best multi‑teacher student saved.
[Epoch 005] train_loss=0.3266
           test_acc=83.29%
  → New best multi‑teacher student saved.
[Epoch 006] train_loss=0.3167
           test_acc=83.27%
[Epoch 007] train_loss=0.3107
           test_acc=83.29%
[Epoch 008] train_loss=0.3016
           test_acc=83.16%
[Epoch 009] train_loss=0.2982
           test_acc=83.42%
  → New best multi‑teacher student saved.
[Epoch 010] train_loss=0.2912
           test_acc=83.29%
[Epoch 011] train_loss=0.2859
           test_acc=83.45%
  → New best multi‑teacher student saved.
[Epoch 012] train_loss=0.2829
      

In [26]:
# —— 单教师蒸馏 —— 取 teacher_models[0] 作为唯一教师
import torch
import torch.nn.functional as F

student = student.to(device)
teacher = teacher_models[0].to(device)
teacher.eval()
for p in teacher.parameters(): p.requires_grad = False

best_acc = 0.0
num_epochs = 200

for epoch in range(1, num_epochs+1):
    student.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        # 1️⃣ 单教师 forward
        with torch.no_grad():
            logits_t = teacher(inputs)                   # [B, C]
            p_soft   = F.softmax(logits_t / tau, dim=1)  # [B, C]

        # 2️⃣ 学生 forward + KD 损失
        logits_s = student(inputs)                      # [B, C]
        s_soft   = F.log_softmax(logits_s / tau, dim=1)
        loss_kl  = F.kl_div(s_soft, p_soft, reduction='batchmean') * (tau*tau)
        loss_ce  = F.cross_entropy(logits_s, targets)
        loss     = alpha * loss_kl + (1 - alpha) * loss_ce

        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    scheduler.step()
    train_loss = running_loss / len(train_loader.dataset)
    print(f"[Single] Epoch {epoch:03d}/{num_epochs}   train_loss={train_loss:.4f}")

    # 3️⃣ 验证
    student.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            preds = student(inputs).argmax(dim=1)
            correct += (preds==targets).sum().item()
            total   += targets.size(0)
    acc = 100.*correct/total
    print(f"          test_acc={acc:.2f}%")

    if acc > best_acc:
        best_acc = acc
        torch.save({
            'epoch':         epoch,
            'student_state': student.state_dict(),
            'optimizer':     optimizer.state_dict(),
            'test_acc':      acc,
        }, "student_single_teacher_best.pth")
        print("  → New best single‑teacher student saved.")


[Single] Epoch 001/200   train_loss=0.5630
          test_acc=82.86%
  → New best single‑teacher student saved.
[Single] Epoch 002/200   train_loss=0.4938
          test_acc=82.48%
[Single] Epoch 003/200   train_loss=0.4694
          test_acc=82.11%
[Single] Epoch 004/200   train_loss=0.4551
          test_acc=82.09%
[Single] Epoch 005/200   train_loss=0.4441
          test_acc=81.96%
[Single] Epoch 006/200   train_loss=0.4323
          test_acc=81.86%
[Single] Epoch 007/200   train_loss=0.4279
          test_acc=81.93%
[Single] Epoch 008/200   train_loss=0.4206
          test_acc=82.08%
[Single] Epoch 009/200   train_loss=0.4127
          test_acc=81.86%
[Single] Epoch 010/200   train_loss=0.4095
          test_acc=81.77%
[Single] Epoch 011/200   train_loss=0.4059
          test_acc=81.87%
[Single] Epoch 012/200   train_loss=0.4009
          test_acc=81.84%
[Single] Epoch 013/200   train_loss=0.4003
          test_acc=81.59%
[Single] Epoch 014/200   train_loss=0.3978
          test_ac

# 下面给出修改后的「单教师蒸馏」那一段，把之前的 teacher_models[0] 全部替换成直接加载 teacher_resnet50_best.pth：

In [21]:
import os, torch
import torch.nn.functional as F
from torchvision.models import resnet50

# ───── 0️⃣ Setup ─────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
save_dir = "checkpoints/single_teacher"
os.makedirs(save_dir, exist_ok=True)

# 假设 student, train_loader, test_loader, optimizer, scheduler, tau, alpha 已提前定义好
student = student.to(device)

# ───── 1️⃣ 把 teacher_resnet50_best.pth 当作唯一教师 ─────
teacher_ckpt = "teacher_resnet50_best.pth"
teacher = resnet50(num_classes=10).to(device)
ck = torch.load(teacher_ckpt, map_location=device)
teacher.load_state_dict(ck['model_state'])
teacher.eval()
for p in teacher.parameters():
    p.requires_grad = False

best_acc = 0.0
num_epochs = 200

# ───── 2️⃣ 蒸馏训练 ─────
for epoch in range(1, num_epochs+1):
    student.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        # —— 教师 forward 取软标签 ——
        with torch.no_grad():
            logits_t = teacher(inputs)                    # [B, C]
            p_soft   = F.softmax(logits_t / tau, dim=1)   # [B, C]

        # —— 学生 forward + KD 损失 ——
        logits_s = student(inputs)                       # [B, C]
        s_soft   = F.log_softmax(logits_s / tau, dim=1)
        loss_kl  = F.kl_div(s_soft, p_soft, reduction='batchmean') * (tau*tau)
        loss_ce  = F.cross_entropy(logits_s, targets)
        loss     = alpha * loss_kl + (1 - alpha) * loss_ce

        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    scheduler.step()
    train_loss = running_loss / len(train_loader.dataset)
    print(f"[Single] Epoch {epoch:03d}/{num_epochs}  train_loss={train_loss:.4f}")

    # —— 验证 ——
    student.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            preds = student(inputs).argmax(dim=1)
            correct += (preds == targets).sum().item()
            total   += targets.size(0)
    acc = 100. * correct / total
    print(f"          test_acc={acc:.2f}%")

    # —— 保存最优 ——
    if acc > best_acc:
        best_acc = acc
        ckpt_name = f"student_single_tau{tau}_alpha{alpha}_best.pth"
        ckpt_path = os.path.join(save_dir, ckpt_name)
        torch.save({
            'epoch':         epoch,
            'student_state': student.state_dict(),
            'optimizer':     optimizer.state_dict(),
            'test_acc':      acc,
            'tau':           tau,
            'alpha':         alpha,
        }, ckpt_path)
        print(f"  → New best single‑teacher student saved to {ckpt_path}")

print(f"\n✅ Single‑teacher distillation complete. Best acc = {best_acc:.2f}%")


[Single] Epoch 001/200  train_loss=3.6901
          test_acc=83.78%
  → New best single‑teacher student saved to checkpoints/single_teacher/student_single_tau5_alpha0.9_best.pth
[Single] Epoch 002/200  train_loss=3.6928
          test_acc=83.87%
  → New best single‑teacher student saved to checkpoints/single_teacher/student_single_tau5_alpha0.9_best.pth
[Single] Epoch 003/200  train_loss=3.6938
          test_acc=83.86%
[Single] Epoch 004/200  train_loss=3.6896
          test_acc=83.84%
[Single] Epoch 005/200  train_loss=3.6929
          test_acc=83.90%
  → New best single‑teacher student saved to checkpoints/single_teacher/student_single_tau5_alpha0.9_best.pth
[Single] Epoch 006/200  train_loss=3.6928
          test_acc=83.93%
  → New best single‑teacher student saved to checkpoints/single_teacher/student_single_tau5_alpha0.9_best.pth
[Single] Epoch 007/200  train_loss=3.6913
          test_acc=83.85%
[Single] Epoch 008/200  train_loss=3.6925
          test_acc=83.78%
[Single] Epoch 0

In [27]:
# —— 硬标签基线 —— 只用 CrossEntropy
import torch

student = student.to(device)
best_acc = 0.0
num_epochs = 200

for epoch in range(1, num_epochs+1):
    student.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        # 1️⃣ 仅学生前向
        logits_s = student(inputs)                   # [B, C]
        loss     = torch.nn.functional.cross_entropy(logits_s, targets)

        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    scheduler.step()
    train_loss = running_loss / len(train_loader.dataset)
    print(f"[Hard ] Epoch {epoch:03d}/{num_epochs}   train_loss={train_loss:.4f}")

    # 2️⃣ 验证
    student.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            preds = student(inputs).argmax(dim=1)
            correct += (preds==targets).sum().item()
            total   += targets.size(0)
    acc = 100.*correct/total
    print(f"          test_acc={acc:.2f}%")

    if acc > best_acc:
        best_acc = acc
        torch.save({
            'epoch':         epoch,
            'student_state': student.state_dict(),
            'optimizer':     optimizer.state_dict(),
            'test_acc':      acc,
        }, "student_hard_label_best.pth")
        print("  → New best hard‑label student saved.")


[Hard ] Epoch 001/200   train_loss=0.3215
          test_acc=81.20%
  → New best hard‑label student saved.
[Hard ] Epoch 002/200   train_loss=0.2625
          test_acc=82.40%
  → New best hard‑label student saved.
[Hard ] Epoch 003/200   train_loss=0.2328
          test_acc=82.57%
  → New best hard‑label student saved.
[Hard ] Epoch 004/200   train_loss=0.2144
          test_acc=82.46%
[Hard ] Epoch 005/200   train_loss=0.1957
          test_acc=82.57%
[Hard ] Epoch 006/200   train_loss=0.1895
          test_acc=83.24%
  → New best hard‑label student saved.
[Hard ] Epoch 007/200   train_loss=0.1810
          test_acc=83.00%
[Hard ] Epoch 008/200   train_loss=0.1696
          test_acc=83.18%
[Hard ] Epoch 009/200   train_loss=0.1654
          test_acc=83.07%
[Hard ] Epoch 010/200   train_loss=0.1545
          test_acc=83.73%
  → New best hard‑label student saved.
[Hard ] Epoch 011/200   train_loss=0.1456
          test_acc=83.44%
[Hard ] Epoch 012/200   train_loss=0.1472
          test_

summary:
Multi‑teacher distillation complete. Best acc: 84.44%
—— 单教师蒸馏 —— 取 teacher_models[0] 作为唯一教师 complete. Best acc: 82.86%
Hard label ‑teacher distillation complete. Best acc:85.51%


# Part6

Explanation:

We loop over each desired teacher‑count
𝐾
K.

For each
(
𝜏
,
𝛼
)
(τ,α) pair, we:

Instantiate a fresh ResNet34 student.

Fuse the first
𝐾
K teachers’
𝜏
τ‑softmax outputs each batch.

Train 200 epochs with KL+CE loss.

Evaluate on the test set and print/store the final accuracy.

All results accumulate in the results list for later comparison or tabulation.

In [17]:
# 只跑之前（Part4）跑过最好的{'τ': 5,  'α': 0.9}✖️「2,4,6」共三个组合
import os, glob
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.models import resnet34, resnet18, resnet50
from torch.utils.data import DataLoader
import pandas as pd

# ───── 0️⃣ Setup device & data loaders ─────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
train_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=True,  download=True, transform=train_transform),
    batch_size=128, shuffle=True,  num_workers=4, pin_memory=True
)
test_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=False, download=True, transform=test_transform),
    batch_size=128, shuffle=False, num_workers=4, pin_memory=True
)

# ───── 1️⃣ Load your pretrained teachers ─────
ckpt_paths = sorted(glob.glob("teacher_*_*_best.pth"))
if not ckpt_paths:
    raise FileNotFoundError("No teacher checkpoints found")
all_teacher_models = []
for path in ckpt_paths:
    arch = os.path.basename(path).split("_")[2]
    ctor = {"resnet18":resnet18,"resnet34":resnet34,"resnet50":resnet50}[arch]
    m = ctor(num_classes=10).to(device)
    ck = torch.load(path, map_location=device)
    m.load_state_dict(ck['model_state'])
    m.eval()
    for p in m.parameters(): p.requires_grad=False
    all_teacher_models.append(m)
print(f"🔍 Loaded {len(all_teacher_models)} teachers")

# ───── 2️⃣ 拿到你之前跑好的 (τ,α) 组合 ─────
rev_results = [
     {'τ': 5,  'α': 0.9},

]
df_rev = pd.DataFrame(rev_results)
combos = df_rev[['τ','α']].to_dict('records')  # list of dicts [{'τ':1,'α':0.1}, ...]

# ───── 3️⃣ 只对 K=2,4,6 做叠加 ─────
Ks = [2,4,6]
num_epochs = 200
final_results = []

for K in Ks:
    teachers = all_teacher_models[:K]
    for combo in combos:
        tau, alpha = combo['τ'], combo['α']

        # fresh student & optimizer/scheduler
        student   = resnet34(num_classes=10).to(device)
        optimizer = torch.optim.SGD(student.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,150], gamma=0.1)

        # training
        for epoch in range(1, num_epochs+1):
            student.train()
            run_loss = 0.0
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                # fuse K teachers
                with torch.no_grad():
                    C = teachers[0](xb).size(1)
                    sum_soft = torch.zeros(xb.size(0), C, device=device)
                    for tm in teachers:
                        sum_soft += F.softmax(tm(xb)/tau, dim=1)
                    p_avg = sum_soft / K
                # distill+CE
                s_logits = student(xb)
                s_soft   = F.log_softmax(s_logits/tau, dim=1)
                l_kl     = F.kl_div(s_soft, p_avg, reduction='batchmean') * (tau*tau)
                l_ce     = F.cross_entropy(s_logits, yb)
                loss     = alpha*l_kl + (1-alpha)*l_ce
                loss.backward()
                optimizer.step()
                run_loss += loss.item()*xb.size(0)
            scheduler.step()
            # 每 50 epochs 打印一次
            if epoch in (1,50,100,150,num_epochs):
                print(f"[K={K} τ={tau} α={alpha}] Epoch {epoch}/{num_epochs} loss={run_loss/len(train_loader.dataset):.4f}")

        # evaluation
        student.eval()
        correct=total=0
        with torch.no_grad():
            for xb,yb in test_loader:
                xb,yb = xb.to(device), yb.to(device)
                preds = student(xb).argmax(1)
                correct += (preds==yb).sum().item()
                total   += yb.size(0)
        acc = 100.*correct/total

        final_results.append({'K':K,'τ':tau,'α':alpha,'test_acc':acc})
        print(f"→ [K={K} τ={tau} α={alpha}] test_acc={acc:.2f}%\n")

# ───── 4️⃣ 保存结果 ─────
df_out = pd.DataFrame(final_results)
df_out.to_csv("multi_teacher_246_sweep.csv", index=False)
print("✅ Done! ")



🔍 Loaded 5 teachers
[K=2 τ=5 α=0.9] Epoch 1/200 loss=7.0123
[K=2 τ=5 α=0.9] Epoch 50/200 loss=0.7304
[K=2 τ=5 α=0.9] Epoch 100/200 loss=0.6725
[K=2 τ=5 α=0.9] Epoch 150/200 loss=0.3162
[K=2 τ=5 α=0.9] Epoch 200/200 loss=0.2322
→ [K=2 τ=5 α=0.9] test_acc=81.18%

[K=4 τ=5 α=0.9] Epoch 1/200 loss=6.6786


KeyboardInterrupt: 

In [18]:
# 只跑之前（Part4）跑过最好的{'τ': 5,  'α': 0.9}✖️「2,4,6」共三个组合
import os, glob
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.models import resnet34, resnet18, resnet50
from torch.utils.data import DataLoader
import pandas as pd

# ───── 0️⃣ Setup device & data loaders ─────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
train_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=True,  download=True, transform=train_transform),
    batch_size=128, shuffle=True,  num_workers=4, pin_memory=True
)
test_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=False, download=True, transform=test_transform),
    batch_size=128, shuffle=False, num_workers=4, pin_memory=True
)

# ───── 1️⃣ Load your pretrained teachers ─────
ckpt_paths = sorted(glob.glob("teacher_*_*_best.pth"))
if not ckpt_paths:
    raise FileNotFoundError("No teacher checkpoints found")
all_teacher_models = []
for path in ckpt_paths:
    arch = os.path.basename(path).split("_")[2]
    ctor = {"resnet18":resnet18,"resnet34":resnet34,"resnet50":resnet50}[arch]
    m = ctor(num_classes=10).to(device)
    ck = torch.load(path, map_location=device)
    m.load_state_dict(ck['model_state'])
    m.eval()
    for p in m.parameters(): p.requires_grad=False
    all_teacher_models.append(m)
print(f"🔍 Loaded {len(all_teacher_models)} teachers")

# ───── 2️⃣ 拿到你之前跑好的 (τ,α) 组合 ─────
rev_results = [
     {'τ': 5,  'α': 0.9},

]
df_rev = pd.DataFrame(rev_results)
combos = df_rev[['τ','α']].to_dict('records')  # list of dicts [{'τ':1,'α':0.1}, ...]

# ───── 3️⃣ 只对 K=2,4,6 做叠加 ─────
Ks = [4,6]
num_epochs = 200
final_results = []

for K in Ks:
    teachers = all_teacher_models[:K]
    for combo in combos:
        tau, alpha = combo['τ'], combo['α']

        # fresh student & optimizer/scheduler
        student   = resnet34(num_classes=10).to(device)
        optimizer = torch.optim.SGD(student.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,150], gamma=0.1)

        # training
        for epoch in range(1, num_epochs+1):
            student.train()
            run_loss = 0.0
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                # fuse K teachers
                with torch.no_grad():
                    C = teachers[0](xb).size(1)
                    sum_soft = torch.zeros(xb.size(0), C, device=device)
                    for tm in teachers:
                        sum_soft += F.softmax(tm(xb)/tau, dim=1)
                    p_avg = sum_soft / K
                # distill+CE
                s_logits = student(xb)
                s_soft   = F.log_softmax(s_logits/tau, dim=1)
                l_kl     = F.kl_div(s_soft, p_avg, reduction='batchmean') * (tau*tau)
                l_ce     = F.cross_entropy(s_logits, yb)
                loss     = alpha*l_kl + (1-alpha)*l_ce
                loss.backward()
                optimizer.step()
                run_loss += loss.item()*xb.size(0)
            scheduler.step()
            # 每 50 epochs 打印一次
            if epoch in (1,50,100,150,num_epochs):
                print(f"[K={K} τ={tau} α={alpha}] Epoch {epoch}/{num_epochs} loss={run_loss/len(train_loader.dataset):.4f}")

        # evaluation
        student.eval()
        correct=total=0
        with torch.no_grad():
            for xb,yb in test_loader:
                xb,yb = xb.to(device), yb.to(device)
                preds = student(xb).argmax(1)
                correct += (preds==yb).sum().item()
                total   += yb.size(0)
        acc = 100.*correct/total

        final_results.append({'K':K,'τ':tau,'α':alpha,'test_acc':acc})
        print(f"→ [K={K} τ={tau} α={alpha}] test_acc={acc:.2f}%\n")

# ───── 4️⃣ 保存结果 ─────
df_out = pd.DataFrame(final_results)
df_out.to_csv("multi_teacher_246_sweep.csv", index=False)
print("✅ Done! ")



🔍 Loaded 5 teachers
[K=4 τ=5 α=0.9] Epoch 1/200 loss=6.1676
[K=4 τ=5 α=0.9] Epoch 50/200 loss=0.5977
[K=4 τ=5 α=0.9] Epoch 100/200 loss=0.5673
[K=4 τ=5 α=0.9] Epoch 150/200 loss=0.2341
[K=4 τ=5 α=0.9] Epoch 200/200 loss=0.1658
→ [K=4 τ=5 α=0.9] test_acc=82.98%

[K=6 τ=5 α=0.9] Epoch 1/200 loss=2.5948
[K=6 τ=5 α=0.9] Epoch 50/200 loss=-2.9111
[K=6 τ=5 α=0.9] Epoch 100/200 loss=-2.9126
[K=6 τ=5 α=0.9] Epoch 150/200 loss=-3.2254
[K=6 τ=5 α=0.9] Epoch 200/200 loss=-3.2822
→ [K=6 τ=5 α=0.9] test_acc=84.07%

✅ Done! 


# 因为上面k=6的时候loss< 0（代码出错，因为只有五个checkpoint，之前却写了六个）所以运行以下代码（k=5）来测试一下。

In [19]:
# 只跑之前（Part4）跑过最好的{'τ': 5,  'α': 0.9}✖️「2,4,6」共三个组合
import os, glob
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.models import resnet34, resnet18, resnet50
from torch.utils.data import DataLoader
import pandas as pd

# ───── 0️⃣ Setup device & data loaders ─────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
train_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=True,  download=True, transform=train_transform),
    batch_size=128, shuffle=True,  num_workers=4, pin_memory=True
)
test_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=False, download=True, transform=test_transform),
    batch_size=128, shuffle=False, num_workers=4, pin_memory=True
)

# ───── 1️⃣ Load your pretrained teachers ─────
ckpt_paths = sorted(glob.glob("teacher_*_*_best.pth"))
if not ckpt_paths:
    raise FileNotFoundError("No teacher checkpoints found")
all_teacher_models = []
for path in ckpt_paths:
    arch = os.path.basename(path).split("_")[2]
    ctor = {"resnet18":resnet18,"resnet34":resnet34,"resnet50":resnet50}[arch]
    m = ctor(num_classes=10).to(device)
    ck = torch.load(path, map_location=device)
    m.load_state_dict(ck['model_state'])
    m.eval()
    for p in m.parameters(): p.requires_grad=False
    all_teacher_models.append(m)
print(f"🔍 Loaded {len(all_teacher_models)} teachers")

# ───── 2️⃣ 拿到你之前跑好的 (τ,α) 组合 ─────
rev_results = [
     {'τ': 5,  'α': 0.9},

]
df_rev = pd.DataFrame(rev_results)
combos = df_rev[['τ','α']].to_dict('records')  # list of dicts [{'τ':1,'α':0.1}, ...]

# ───── 3️⃣ 只对 K=5 做叠加 ─────
Ks = [5]
num_epochs = 200
final_results = []

for K in Ks:
    teachers = all_teacher_models[:K]
    for combo in combos:
        tau, alpha = combo['τ'], combo['α']

        # fresh student & optimizer/scheduler
        student   = resnet34(num_classes=10).to(device)
        optimizer = torch.optim.SGD(student.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,150], gamma=0.1)

        # training
        for epoch in range(1, num_epochs+1):
            student.train()
            run_loss = 0.0
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                # fuse K teachers
                with torch.no_grad():
                    C = teachers[0](xb).size(1)
                    sum_soft = torch.zeros(xb.size(0), C, device=device)
                    for tm in teachers:
                        sum_soft += F.softmax(tm(xb)/tau, dim=1)
                    p_avg = sum_soft / K
                # distill+CE
                s_logits = student(xb)
                s_soft   = F.log_softmax(s_logits/tau, dim=1)
                l_kl     = F.kl_div(s_soft, p_avg, reduction='batchmean') * (tau*tau)
                l_ce     = F.cross_entropy(s_logits, yb)
                loss     = alpha*l_kl + (1-alpha)*l_ce
                loss.backward()
                optimizer.step()
                run_loss += loss.item()*xb.size(0)
            scheduler.step()
            # 每 50 epochs 打印一次
            if epoch in (1,50,100,150,num_epochs):
                print(f"[K={K} τ={tau} α={alpha}] Epoch {epoch}/{num_epochs} loss={run_loss/len(train_loader.dataset):.4f}")

        # evaluation
        student.eval()
        correct=total=0
        with torch.no_grad():
            for xb,yb in test_loader:
                xb,yb = xb.to(device), yb.to(device)
                preds = student(xb).argmax(1)
                correct += (preds==yb).sum().item()
                total   += yb.size(0)
        acc = 100.*correct/total

        final_results.append({'K':K,'τ':tau,'α':alpha,'test_acc':acc})
        print(f"→ [K={K} τ={tau} α={alpha}] test_acc={acc:.2f}%\n")

# ───── 4️⃣ 保存结果 ─────
df_out = pd.DataFrame(final_results)
df_out.to_csv("multi_teacher_246_sweep.csv", index=False)
print("✅ Done! ")



🔍 Loaded 5 teachers
[K=5 τ=5 α=0.9] Epoch 1/200 loss=7.7488
[K=5 τ=5 α=0.9] Epoch 50/200 loss=0.6674
[K=5 τ=5 α=0.9] Epoch 100/200 loss=0.6132
[K=5 τ=5 α=0.9] Epoch 150/200 loss=0.2405
[K=5 τ=5 α=0.9] Epoch 200/200 loss=0.1672
→ [K=5 τ=5 α=0.9] test_acc=83.78%

✅ Done! 


In [ ]:
# Part 6 遍历所有36个超参组合
import os, glob
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.models import resnet34, resnet18, resnet50
from torch.utils.data import DataLoader
import pandas as pd

# ───── 0️⃣ Setup device & data loaders ─────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
train_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=True,  download=True, transform=train_transform),
    batch_size=128, shuffle=True,  num_workers=4, pin_memory=True
)
test_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=False, download=True, transform=test_transform),
    batch_size=128, shuffle=False, num_workers=4, pin_memory=True
)

# ───── 1️⃣ Load your pretrained teachers ─────
ckpt_paths = sorted(glob.glob("teacher_*_*_best.pth"))
if not ckpt_paths:
    raise FileNotFoundError("No teacher checkpoints found")
all_teacher_models = []
for path in ckpt_paths:
    arch = os.path.basename(path).split("_")[2]
    ctor = {"resnet18":resnet18,"resnet34":resnet34,"resnet50":resnet50}[arch]
    m = ctor(num_classes=10).to(device)
    ck = torch.load(path, map_location=device)
    m.load_state_dict(ck['model_state'])
    m.eval()
    for p in m.parameters(): p.requires_grad=False
    all_teacher_models.append(m)
print(f"🔍 Loaded {len(all_teacher_models)} teachers")

# ───── 2️⃣ 拿到你之前跑好的 (τ,α) 组合 ─────
rev_results = [
    {'τ': 1,  'α': 0.1}, {'τ': 1,  'α': 0.5}, {'τ': 1,  'α': 0.9},
    {'τ': 5,  'α': 0.1}, {'τ': 5,  'α': 0.5}, {'τ': 5,  'α': 0.9},
    {'τ': 10, 'α': 0.1}, {'τ': 10, 'α': 0.5}, {'τ': 10, 'α': 0.9},
    {'τ': 20, 'α': 0.1}, {'τ': 20, 'α': 0.5}, {'τ': 20, 'α': 0.9},
]
df_rev = pd.DataFrame(rev_results)
combos = df_rev[['τ','α']].to_dict('records')  # list of dicts [{'τ':1,'α':0.1}, ...]

# ───── 3️⃣ 只对 K=2,4,6 做叠加 ─────
Ks = [2,4,6]
num_epochs = 200
final_results = []

for K in Ks:
    teachers = all_teacher_models[:K]
    for combo in combos:
        tau, alpha = combo['τ'], combo['α']

        # fresh student & optimizer/scheduler
        student   = resnet34(num_classes=10).to(device)
        optimizer = torch.optim.SGD(student.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,150], gamma=0.1)

        # training
        for epoch in range(1, num_epochs+1):
            student.train()
            run_loss = 0.0
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                # fuse K teachers
                with torch.no_grad():
                    C = teachers[0](xb).size(1)
                    sum_soft = torch.zeros(xb.size(0), C, device=device)
                    for tm in teachers:
                        sum_soft += F.softmax(tm(xb)/tau, dim=1)
                    p_avg = sum_soft / K
                # distill+CE
                s_logits = student(xb)
                s_soft   = F.log_softmax(s_logits/tau, dim=1)
                l_kl     = F.kl_div(s_soft, p_avg, reduction='batchmean') * (tau*tau)
                l_ce     = F.cross_entropy(s_logits, yb)
                loss     = alpha*l_kl + (1-alpha)*l_ce
                loss.backward()
                optimizer.step()
                run_loss += loss.item()*xb.size(0)
            scheduler.step()
            # 每 50 epochs 打印一次
            if epoch in (1,50,100,150,num_epochs):
                print(f"[K={K} τ={tau} α={alpha}] Epoch {epoch}/{num_epochs} loss={run_loss/len(train_loader.dataset):.4f}")

        # evaluation
        student.eval()
        correct=total=0
        with torch.no_grad():
            for xb,yb in test_loader:
                xb,yb = xb.to(device), yb.to(device)
                preds = student(xb).argmax(1)
                correct += (preds==yb).sum().item()
                total   += yb.size(0)
        acc = 100.*correct/total

        final_results.append({'K':K,'τ':tau,'α':alpha,'test_acc':acc})
        print(f"→ [K={K} τ={tau} α={alpha}] test_acc={acc:.2f}%\n")

# ───── 4️⃣ 保存结果 ─────
df_out = pd.DataFrame(final_results)
df_out.to_csv("multi_teacher_246_sweep.csv", index=False)
print("✅ Done! ")



In [ ]:
# Part 6 遍历所有36个超参组合
import os, glob
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.models import resnet18, resnet34, resnet50
from torch.utils.data import DataLoader
from itertools import product
import pandas as pd

# ───── 0️⃣ Setup device & data loaders ─────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# CIFAR‑10 transforms (same as in code 4)
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

batch_size = 128
train_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=True,  download=True, transform=train_transform),
    batch_size=batch_size, shuffle=True,  num_workers=4, pin_memory=True
)
test_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=False, download=True, transform=test_transform),
    batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True
)

# ───── 1️⃣ Build all_teacher_models from your code 3 checkpoints ─────
ckpt_paths = sorted(glob.glob("teacher_*_*_best.pth"))
if not ckpt_paths:
    raise FileNotFoundError("No teacher checkpoints found matching teacher_*_*_best.pth")

all_teacher_models = []
for path in ckpt_paths:
    arch = os.path.basename(path).split("_")[2]  # e.g. 'resnet34'
    if   arch=="resnet18": ctor = resnet18
    elif arch=="resnet34": ctor = resnet34
    elif arch=="resnet50": ctor = resnet50
    else: raise ValueError(f"Unexpected arch {arch} in {path}")
    m = ctor(num_classes=10).to(device)
    ck = torch.load(path, map_location=device)
    m.load_state_dict(ck['model_state'])
    m.eval()
    for p in m.parameters(): p.requires_grad = False
    all_teacher_models.append(m)

print(f"🔍  Loaded {len(all_teacher_models)} teachers from disk")

# ───── 2️⃣ Hyperparameters & sweep setup ─────
taus       = [1, 5, 10, 20]
alphas     = [0.1, 0.5, 0.9]
Ks         = [2, 4, 8]
num_epochs = 200

results = []

# ───── 3️⃣ Multi‑teacher KD sweep ─────
for K in Ks:
    teachers = all_teacher_models[:K]
    for tau, alpha in product(taus, alphas):
        # fresh student + optimizer/scheduler
        student   = resnet34(num_classes=10).to(device)
        optimizer = torch.optim.SGD(
            student.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=[100,150], gamma=0.1
        )

        # training loop
        for epoch in range(1, num_epochs+1):
            student.train()
            run_loss = 0.0

            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                # fuse K teachers
                with torch.no_grad():
                    C = teachers[0](xb).size(1)
                    sum_soft = torch.zeros(xb.size(0), C, device=device)
                    for tm in teachers:
                        sum_soft += F.softmax(tm(xb)/tau, dim=1)
                    p_avg = sum_soft / K
                # student forward + loss
                s_logits = student(xb)
                s_soft   = F.log_softmax(s_logits/tau, dim=1)
                l_kl     = F.kl_div(s_soft, p_avg, reduction='batchmean') * (tau*tau)
                l_ce     = F.cross_entropy(s_logits, yb)
                loss     = alpha*l_kl + (1-alpha)*l_ce
                loss.backward()
                optimizer.step()
                run_loss += loss.item()*xb.size(0)

            scheduler.step()

            # ── print periodic epoch info ──
            if epoch == 1 or epoch == num_epochs or epoch % 50 == 0:
                avg_loss = run_loss / len(train_loader.dataset)
                print(f"[K={K}, τ={tau}, α={alpha}] Epoch {epoch:03d}/{num_epochs} ― train_loss={avg_loss:.4f}")

        # evaluation
        student.eval()
        correct = total = 0
        with torch.no_grad():
            for xb,yb in test_loader:
                xb,yb = xb.to(device), yb.to(device)
                preds = student(xb).argmax(1)
                correct += (preds==yb).sum().item()
                total   += yb.size(0)
        acc = 100.*correct/total

        results.append({'K':K, 'tau':tau, 'alpha':alpha, 'test_acc':acc})
        print(f"→ K={K}, τ={tau}, α={alpha}  →  test_acc={acc:.2f}%\n")

# ───── Done ─────
df_results = pd.DataFrame(results)
df_results.to_csv("multi_teacher_sweep.csv", index=False)
print("✅ Sweep complete, saved to multi_teacher_sweep.csv")


# 新的 code 6
新的 code 6 正是在新的 code 3（把强化训练后的 teacher checkpoints 存到 `checkpoints/single_teacher/`）的基础上，来检验用「真正变强」的 teacher 模型去蒸馏对 student 的效果有多大提升。换句话说：

1. **新的 code 3** 让你的 ResNet‑50 teacher 真正跑到 ∼90%+ 的水平，并把 checkpoint 全都存到 `checkpoints/single_teacher/` 目录下，和旧的 `teacher_*_*_best.pth` 不会互相覆盖。  
2. **rerun code 6** 就是在这批真实好用的 teacher 上，用先前在 Part 4 找到的最佳 \{τ=5, α=0.9\}，分别测试 K=2,4,6 三种老师数量对 student 最终精度的影响。  
3. 这样做的意义就是：  
   - **不用再跑 36 个** (K×τ×α) 组合，直接锁定最有可能打赢硬标签的「黄金超参」；  
   - 确保蒸馏所用的 teacher 真的是你刚刚 mixup+label‐smoothing 训练出来、在验证集跑到 ∼89–90%+ 的那几位，而不是旧 teacher；  
   - 快速对比：K=2、4、6 这三个点，student 能提升到什么水平，从而判断要不要大规模扩充 teacher 数量。  

如果你不把 code 6 重新跑一遍，它只会加载旧有的、只有 ∼76% 验证精度的 teachers──那当然拿不到好的 student 蒸馏效果。  

所以 **下一步** 就是用新的、90%+ 的 teacher checkpoints，跑一遍 code 6，看看在 \{τ=5,α=0.9\} 下 K=2、4、6 时，student 最终能到几几％，再决定要不要把 K 再往上拉，或者混入不同超参做更细致的 sweep。

In [22]:
import os, glob
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.models import resnet34, resnet18, resnet50
from torch.utils.data import DataLoader
import pandas as pd

# ───── 0️⃣ Setup device & data loaders ─────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
train_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=True, download=True, transform=train_transform),
    batch_size=128, shuffle=True, num_workers=4, pin_memory=True
)
test_loader = DataLoader(
    datasets.CIFAR10(root='data/', train=False, download=True, transform=test_transform),
    batch_size=128, shuffle=False, num_workers=4, pin_memory=True
)

# ───── 1️⃣ Load pretrained teachers ─────
ckpt_paths = sorted(glob.glob("teacher_*_*_best.pth"))
if not ckpt_paths:
    raise FileNotFoundError("No teacher checkpoints found")
all_teacher_models = []
for path in ckpt_paths:
    arch = os.path.basename(path).split("_")[2]  # 'resnet34' 等
    ctor = {"resnet18": resnet18, "resnet34": resnet34, "resnet50": resnet50}[arch]
    m = ctor(num_classes=10).to(device)
    ck = torch.load(path, map_location=device)
    m.load_state_dict(ck['model_state'])
    m.eval()
    for p in m.parameters(): p.requires_grad = False
    all_teacher_models.append(m)
print(f"🔍 Loaded {len(all_teacher_models)} teachers")

# ───── 2️⃣ 固定 (τ,α) 仅这一组 ─────
tau, alpha = 5, 0.9

# ───── 3️⃣ 只对 K = 2,4,6 做多教师蒸馏 ─────
Ks = [2, 4, 6]
num_epochs = 200
final_results = []

for K in Ks:
    teachers = all_teacher_models[:K]
    print(f"\n=== Multi‑teacher KD with K={K}, τ={tau}, α={alpha} ===")
    # 注意：在每个 K 下都要 fresh 一个 student
    student = None
    for epoch in [1, 50, 100, 150, num_epochs]:
        # 第一次循环前才 new model
        if student is None:
            student = resnet34(num_classes=10).to(device)
            optimizer = torch.optim.SGD(
                student.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4
            )
            scheduler = torch.optim.lr_scheduler.MultiStepLR(
                optimizer, milestones=[100,150], gamma=0.1
            )

        # 从上次的 epoch+1 跑到当前 epoch
        start = 1 if epoch==1 else prev_epoch+1
        run_loss = 0.0
        student.train()
        for e in range(start, epoch+1):
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                # fuse K teachers
                with torch.no_grad():
                    C = teachers[0](xb).size(1)
                    sum_soft = torch.zeros(xb.size(0), C, device=device)
                    for tm in teachers:
                        sum_soft += F.softmax(tm(xb)/tau, dim=1)
                    p_avg = sum_soft / K
                # distill + CE
                logits_s = student(xb)
                s_soft   = F.log_softmax(logits_s/tau, dim=1)
                l_kl     = F.kl_div(s_soft, p_avg, reduction='batchmean') * (tau*tau)
                l_ce     = F.cross_entropy(logits_s, yb)
                loss     = alpha*l_kl + (1-alpha)*l_ce
                loss.backward()
                optimizer.step()
                run_loss += loss.item()*xb.size(0)
            scheduler.step()

        prev_epoch = epoch
        avg_loss = run_loss / len(train_loader.dataset)
        print(f"[K={K} τ={tau} α={alpha}] Epoch {epoch}/{num_epochs}  train_loss={avg_loss:.4f}")

    # ── 最终评估 ──
    student.eval()
    correct = total = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = student(xb).argmax(1)
            correct += (preds==yb).sum().item()
            total   += yb.size(0)
    acc = 100.*correct/total

    final_results.append({'K':K, 'τ':tau, 'α':alpha, 'test_acc':acc})
    print(f"→ [K={K} τ={tau} α={alpha}] test_acc={acc:.2f}%\n")

# ───── 4️⃣ 保存结果 ─────
df_out = pd.DataFrame(final_results)
df_out.to_csv("multi_teacher_246_only.csv", index=False)
print("✅ Done! Results saved to multi_teacher_246_only.csv")


🔍 Loaded 5 teachers

=== Multi‑teacher KD with K=2, τ=5, α=0.9 ===
[K=2 τ=5 α=0.9] Epoch 1/200  train_loss=5.9960
[K=2 τ=5 α=0.9] Epoch 50/200  train_loss=44.7222
[K=2 τ=5 α=0.9] Epoch 100/200  train_loss=31.8692
[K=2 τ=5 α=0.9] Epoch 150/200  train_loss=14.3353
[K=2 τ=5 α=0.9] Epoch 200/200  train_loss=10.4413
→ [K=2 τ=5 α=0.9] test_acc=81.04%


=== Multi‑teacher KD with K=4, τ=5, α=0.9 ===
[K=4 τ=5 α=0.9] Epoch 1/200  train_loss=6.2748
[K=4 τ=5 α=0.9] Epoch 50/200  train_loss=46.6638
[K=4 τ=5 α=0.9] Epoch 100/200  train_loss=29.6004
[K=4 τ=5 α=0.9] Epoch 150/200  train_loss=12.4816
[K=4 τ=5 α=0.9] Epoch 200/200  train_loss=8.9397
→ [K=4 τ=5 α=0.9] test_acc=83.03%


=== Multi‑teacher KD with K=6, τ=5, α=0.9 ===
[K=6 τ=5 α=0.9] Epoch 1/200  train_loss=2.3555
[K=6 τ=5 α=0.9] Epoch 50/200  train_loss=-128.3479
[K=6 τ=5 α=0.9] Epoch 100/200  train_loss=-145.7046
[K=6 τ=5 α=0.9] Epoch 150/200  train_loss=-160.6823
[K=6 τ=5 α=0.9] Epoch 200/200  train_loss=-163.7050
→ [K=6 τ=5 α=0.9] test_a

# part7：Results Visualization & Analysis

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# 1. Load the three CSVs into DataFrames
for fn in ("baseline.csv","single_teacher.csv","multi_teacher.csv"):
    if not os.path.isfile(fn):
        raise FileNotFoundError(f"Required file not found: {fn}")

df_base   = pd.read_csv("baseline.csv"       ).rename(columns={'test_acc':'acc_base'})
df_single = pd.read_csv("single_teacher.csv" ).rename(columns={'test_acc':'acc_single'})
df_multi  = pd.read_csv("multi_teacher.csv"  ).rename(columns={'test_acc':'acc_multi'})

# 2. Merge on the grid indices (tau, alpha, K)
df = df_base.merge(df_single, on=['tau','alpha','K']) \
            .merge(df_multi,  on=['tau','alpha','K'])

# 3. Compute Δ‑columns
df['Δ_single'] = df['acc_single'] - df['acc_base']
df['Δ_multi']  = df['acc_multi']  - df['acc_base']

# 4. Quantitative tables
print("\n=== Full Accuracy Comparison ===")
print(df[['tau','alpha','K','acc_base','acc_single','acc_multi']]
      .sort_values(['K','alpha','tau'])
      .to_string(index=False))

print("\n=== Δ-Single‑Teacher Accuracy Pivot ===")
pivot_single = df.pivot_table(index=['tau','alpha'], columns='K', values='Δ_single')
print(pivot_single)

print("\n=== Δ-Multi‑Teacher Accuracy Pivot ===")
pivot_multi  = df.pivot_table(index=['tau','alpha'], columns='K', values='Δ_multi')
print(pivot_multi)

# 5. Save combined table (optional)
df.to_csv("combined_results.csv", index=False)


# 6. Visualization: Accuracy vs Temperature for each (α, K)
for alpha in sorted(df['alpha'].unique()):
    for K in sorted(df['K'].unique()):
        sub = df[(df.alpha==alpha)&(df.K==K)].sort_values('tau')
        plt.figure()
        plt.plot(sub.tau, sub.acc_base,   'k--',   label='Hard‑label baseline')
        plt.plot(sub.tau, sub.acc_single, 'b-o',   label='Single‑teacher KD')
        plt.plot(sub.tau, sub.acc_multi,  'r-s',   label=f'Multi‑teacher KD (K={K})')
        plt.title(f'Accuracy vs τ  (α={alpha}, K={K})')
        plt.xlabel('Temperature τ')
        plt.ylabel('Test Accuracy (%)')
        plt.legend()
        plt.grid(True)
        plt.savefig(f"acc_vs_tau_alpha{alpha}_K{K}.png")
        plt.close()

# 7. Visualization: Δ-Multi vs K at best (τ, α)
best_idx    = df['acc_multi'].idxmax()
best_tau    = df.at[best_idx, 'tau']
best_alpha  = df.at[best_idx, 'alpha']
best_slice  = df[(df.tau==best_tau)&(df.alpha==best_alpha)].sort_values('K')

plt.figure()
plt.bar(best_slice.K.astype(str), best_slice['Δ_multi'], color='skyblue')
plt.title(f'Δ‑Multi‑Teacher Accuracy vs K  (τ={best_tau}, α={best_alpha})')
plt.xlabel('Number of Teachers K')
plt.ylabel('Δ‑Accuracy (%)')
plt.grid(axis='y')
plt.savefig("delta_vs_K.png")
plt.close()

print(f"\nPlotted:\n - Accuracy vs τ curves for each (α,K)\n - Δ‑Accuracy vs K at best τ={best_tau}, α={best_alpha}\n")
